# Real-Time Fraud Detection Monitoring Dashboard

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import os
import glob

plt.style.use('seaborn-v0_8')
%matplotlib inline

In [ ]:
def load_log_data():
    """Load real-time logs"""
    alerts_data = []
    stats_data = []
    
    # Load alerts
    if os.path.exists('../logs/real_time_alerts.jsonl'):
        with open('../logs/real_time_alerts.jsonl', 'r') as f:
            for line in f:
                alerts_data.append(json.loads(line))
    
    # Load stats
    if os.path.exists('../logs/real_time_stats.jsonl'):
        with open('../logs/real_time_stats.jsonl', 'r') as f:
            for line in f:
                stats_data.append(json.loads(line))
    
    # Load batch alerts from streaming
    batch_files = glob.glob('../logs/fraud_alerts_batch_*.json')
    batch_alerts = []
    for file in batch_files:
        with open(file, 'r') as f:
            batch_data = json.load(f)
            batch_alerts.extend(batch_data.get('cases', []))
    
    return pd.DataFrame(alerts_data), pd.DataFrame(stats_data), pd.DataFrame(batch_alerts)

alerts_df, stats_df, batch_alerts_df = load_log_data()

print(f"Real-time alerts: {len(alerts_df)}")
print(f"Batch statistics: {len(stats_df)}")
print(f"Streaming alerts: {len(batch_alerts_df)}")

In [ ]:
# Create sample data if logs are empty
if len(alerts_df) == 0:
    # Generate sample alert data
    sample_alerts = []
    for i in range(20):
        sample_alerts.append({
            'timestamp': (datetime.now() - timedelta(hours=i)).isoformat(),
            'transaction_id': f'TXN_{i:04d}',
            'amount': np.random.exponential(200),
            'confidence': np.random.uniform(0.6, 0.95),
            'iso_score': np.random.uniform(-0.5, -0.1)
        })
    alerts_df = pd.DataFrame(sample_alerts)

if len(stats_df) == 0:
    # Generate sample stats data
    sample_stats = []
    for i in range(10):
        sample_stats.append({
            'batch_id': i,
            'timestamp': (datetime.now() - timedelta(minutes=i*5)).isoformat(),
            'transactions_processed': np.random.randint(80, 120),
            'fraud_detected': np.random.randint(0, 8),
            'fraud_rate': np.random.uniform(0.01, 0.08)
        })
    stats_df = pd.DataFrame(sample_stats)

print("Sample data created for demonstration")

In [ ]:
# Real-time monitoring dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Fraud Detection Over Time', 'Alert Confidence Distribution', 
                   'Transaction Amounts', 'Fraud Rate by Batch'),
    specs=[[{"secondary_y": True}, {}],
           [{}, {}]]
)

# Convert timestamps
if len(alerts_df) > 0:
    alerts_df['timestamp'] = pd.to_datetime(alerts_df['timestamp'])
if len(stats_df) > 0:
    stats_df['timestamp'] = pd.to_datetime(stats_df['timestamp'])

# Plot 1: Fraud detection over time
if len(stats_df) > 0:
    fig.add_trace(
        go.Scatter(x=stats_df['timestamp'], y=stats_df['fraud_detected'],
                  mode='lines+markers', name='Fraud Detected',
                  line=dict(color='red')),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(x=stats_df['timestamp'], y=stats_df['transactions_processed'],
                  mode='lines+markers', name='Total Processed',
                  line=dict(color='blue'), yaxis='y2'),
        row=1, col=1, secondary_y=True
    )

# Plot 2: Confidence distribution
if len(alerts_df) > 0:
    fig.add_trace(
        go.Histogram(x=alerts_df['confidence'], nbinsx=20,
                    name='Confidence Distribution',
                    marker_color='orange'),
        row=1, col=2
    )

# Plot 3: Transaction amounts
if len(alerts_df) > 0:
    fig.add_trace(
        go.Scatter(x=alerts_df['timestamp'], y=alerts_df['amount'],
                  mode='markers', name='Fraud Amounts',
                  marker=dict(size=8, color='red', opacity=0.7)),
        row=2, col=1
    )

# Plot 4: Fraud rate by batch
if len(stats_df) > 0:
    fig.add_trace(
        go.Bar(x=stats_df['batch_id'], y=stats_df['fraud_rate'],
              name='Fraud Rate', marker_color='darkred'),
        row=2, col=2
    )

fig.update_layout(height=800, title_text="Real-Time Fraud Detection Dashboard")
fig.show()

# Save dashboard
fig.write_html('../reports/fraud_detection_dashboard.html')

In [ ]:
# Summary statistics
print("=== FRAUD DETECTION SUMMARY ===")

if len(alerts_df) > 0:
    print(f"Total Fraud Alerts: {len(alerts_df)}")
    print(f"Average Confidence: {alerts_df['confidence'].mean():.3f}")
    print(f"Average Fraud Amount: ${alerts_df['amount'].mean():.2f}")
    print(f"Max Fraud Amount: ${alerts_df['amount'].max():.2f}")

if len(stats_df) > 0:
    total_processed = stats_df['transactions_processed'].sum()
    total_fraud = stats_df['fraud_detected'].sum()
    overall_fraud_rate = total_fraud / total_processed if total_processed > 0 else 0
    
    print(f"\nTotal Transactions Processed: {total_processed}")
    print(f"Total Fraud Detected: {total_fraud}")
    print(f"Overall Fraud Rate: {overall_fraud_rate:.3f} ({overall_fraud_rate*100:.2f}%)")

print("\n=== SYSTEM STATUS ===")
print("✅ Real-time detection: ACTIVE")
print("✅ Alert logging: ACTIVE")
print("✅ Dashboard: UPDATED")

In [ ]:
# Recent alerts table
if len(alerts_df) > 0:
    print("\n=== RECENT FRAUD ALERTS ===")
    recent_alerts = alerts_df.sort_values('timestamp', ascending=False).head(10)
    display_cols = ['timestamp', 'transaction_id', 'amount', 'confidence']
    
    for col in display_cols:
        if col in recent_alerts.columns:
            if col == 'amount':
                recent_alerts[col] = recent_alerts[col].apply(lambda x: f"${x:.2f}")
            elif col == 'confidence':
                recent_alerts[col] = recent_alerts[col].apply(lambda x: f"{x:.3f}")
    
    print(recent_alerts[display_cols].to_string(index=False))

print("\n📊 Dashboard saved to: ../reports/fraud_detection_dashboard.html")
print("🔄 Refresh this notebook to see updated real-time data")